In [ ]:
import glob
import os
import sys
import argparse
import time
from datetime import datetime
import random
import numpy as np
from matplotlib import cm
import open3d as o3d
import carla
import math

import cv2

import sys
import weakref

PREFERRED_SPEED = 40 # what it says
SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the

def maintain_speed(s):
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 

collision_detected = False

def collision_callback(event):
	global collision_detected
	collision_detected = True

client = carla.Client('localhost', 2000)
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

start_point = spawn_points[0]

desired_rotation = carla.Rotation(pitch=0.0, yaw=90.0, roll=0.0)
new_spawn_transform2 = carla.Transform(
        carla.Location(
            start_point.location.x + 15,
            start_point.location.y - 10,
            start_point.location.z
        ),
        desired_rotation
)

blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('*model3*')[0]
vehicle = world.spawn_actor(vehicle_bp, new_spawn_transform2)

collision_sensor = blueprint_library.find("sensor.other.collision")
collision_sensor = world.spawn_actor(collision_sensor, carla.Transform(), attach_to=vehicle)
collision_sensor.listen(lambda event: collision_callback(event))

while True:
	world.tick()
	steering_angle = 0
	v = vehicle.get_velocity()
	speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2), 0)
	estimated_throttle = maintain_speed(speed)
	vehicle_control = carla.VehicleControl(throttle=estimated_throttle, steer=steering_angle)
	vehicle.apply_control(vehicle_control)
	if collision_detected == True:
		collision_detected = False
		vehicle.destroy()
		collision_sensor.stop()
		collision_sensor.destroy()
		vehicle = world.spawn_actor(vehicle_bp, new_spawn_transform2)

		collision_sensor = blueprint_library.find("sensor.other.collision")
		collision_sensor = world.spawn_actor(collision_sensor, carla.Transform(), attach_to=vehicle)
		collision_sensor.listen(lambda event: collision_callback(event))




In [ ]:
for sensor in world.get_actors().filter('*sensor*'):
	sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()